In [1]:
# This script takes as input geo_interlinks enriched with mappings JL - Geonames
# and enriches the data with a third link to wiki/dbpedia
import pprint, time, requests
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import RDF, OWL, RDFS
import urllib.parse
GEONAMES_USER = ""

gn = Namespace("http://www.geonames.org/ontology#")
gn_root = URIRef("http://sws.geonames.org/")
dataset = Graph()
dataset.bind("owl", OWL)

In [2]:
sparql = SPARQLWrapper("http://data.judaicalink.org/sparql/query")
sparql.setQuery("""
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    SELECT ?s
    WHERE { GRAPH ?g {
        ?s ?p ?o
        FILTER(strstarts(str(?s), "http://data.judaicalink.org/data/geonames/" ))
        }}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [3]:
jl_geo_uris = list(set([res['s']['value'] for res in results['results']['bindings']]))

In [ ]:
temp = set([])
print(len(temp))
c = 0
for resource in jl_geo_uris:
    c += 1
    # http://data.judaicalink.org/data/geonames/12345 --> 12345
    query = resource.split('/')[-1]
    print("|", end="")
    url = "https://sws.geonames.org/"+query+"/about.rdf"
    response = requests.get(url)
    response_rdf = Graph()
    #print(response.status_code, end=" ")
    if response.status_code == 200:
        data = response_rdf.parse(data=response.text, format='xml')
        
        for s,p,o in data.triples((None, gn.wikipediaArticle, None)):
            temp.add((resource, urllib.parse.unquote(o)))
            #print(urllib.parse.unquote(o))
            
        for s,p,o in data.triples((None, RDFS.seeAlso, None)):
            if 'dbpedia' in o:
                temp.add((resource, urllib.parse.unquote(o)))
                #print(urllib.parse.unquote(o))
    time.sleep(3)
    if c % 100 == 0:
        print(c)

0
|||||||||||||||||||||||||||||

In [10]:
for (s,o) in temp:
    dataset.add((URIRef(s), OWL.sameAs, URIRef(o)))

In [11]:
dataset.serialize(destination="geo_wiki_dbpedia.ttl", format='ttl')